In [2]:
# Basic convolutional neural network for MNIST digit
# Steven Su
# Compiled from multiple sources including 
# 1. https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-from-scratch-for-mnist-handwritten-digit-classification/
# and 2. https://www.kaggle.com/code/elcaiseri/mnist-simple-cnn-keras-accuracy-0-99-top-1/notebook

# Using https://www.kaggle.com/code/cdeotte/how-to-choose-cnn-architecture-mnist/notebook#Experiment-4

from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import SGD

In [3]:
# load train and test dataset
def load_dataset():
    # load dataset
    (trainX, trainY), (testX, testY) = mnist.load_data()
    # reshape dataset to have a single channel
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))
    # one hot encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY
 
# scale pixels
def prep_pixels(train, test):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm

def define_model():
    model = Sequential()

    model.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(32,kernel_size=3,activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(64,kernel_size=3,activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,kernel_size=3,activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model

tx, ty, vx, vy = load_dataset()
tx_norm, vx_norm = prep_pixels(tx,vx)

In [5]:
epochs = 35
batch_size = 64
train_steps = tx_norm.shape[0] // batch_size
valid_steps = vx_norm.shape[0] // batch_size

model = define_model()

history = model.fit(tx_norm, ty, epochs=epochs, batch_size=batch_size, validation_data=(vx_norm, vy), steps_per_epoch = train_steps, validation_steps = valid_steps)

2022-05-18 12:11:47.976315: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-18 12:11:47.976633: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 12:11:47.976795: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 12:11:47.976885: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Epoch 1/35


2022-05-18 12:11:49.458569: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201
2022-05-18 12:11:49.743764: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


937/937 [==============================] - 16s 15ms/step - loss: 0.2182 - accuracy: 0.9337 - val_loss: 0.0401 - val_accuracy: 0.9858
Epoch 2/35
937/937 [==============================] - 13s 14ms/step - loss: 0.0668 - accuracy: 0.9797 - val_loss: 0.0349 - val_accuracy: 0.9886
Epoch 3/35
937/937 [==============================] - 14s 15ms/step - loss: 0.0519 - accuracy: 0.9842 - val_loss: 0.0376 - val_accuracy: 0.9884
Epoch 4/35
937/937 [==============================] - 13s 14ms/step - loss: 0.0428 - accuracy: 0.9871 - val_loss: 0.0225 - val_accuracy: 0.9929
Epoch 5/35
937/937 [==============================] - 13s 13ms/step - loss: 0.0399 - accuracy: 0.9878 - val_loss: 0.0213 - val_accuracy: 0.9928
Epoch 6/35
937/937 [==============================] - 13s 14ms/step - loss: 0.0365 - accuracy: 0.9890 - val_loss: 0.0227 - val_accuracy: 0.9930
Epoch 7/35
937/937 [==============================] - 13s 14ms/step - loss: 0.0307 - accuracy: 0.9902 - val_loss: 0.0217 - val_accuracy: 0.9938
Epo

In [6]:
import tensorflow as tf
tf.keras.models.save_model(
    model,
    "/home/stevensu/Documents/Github/Model2",
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

2022-05-18 12:19:54.392837: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/stevensu/Documents/Github/Model2/assets


In [4]:
import tensorflow as tf
tf.config.list_physical_devices('GPU') 

2022-05-18 12:11:41.409392: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 12:11:41.413733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 12:11:41.413892: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]